## ULMFiT model for Twitter US Airlines Sentiment 

In [0]:
# Import necessary libraries.
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [0]:
# We disable warnings & only show errors of TensorFlow.
tf.logging.set_verbosity(tf.logging.ERROR)

Now read **Twitter.csv** file as a Pandas dataframe, randomize data & see first 10 records of features, targets from dataframe.

In [0]:
# Read data in CSV file as a Pandas DataFrame.
twitter_df = pd.read_csv('Tweets.csv', sep=',')

# Now randomize data using permutation to avoid trouble if the data is in some sorted order.
twitter_df = twitter_df.reindex(np.random.permutation(twitter_df.index))

# Take a look at our features (i.e. 'text' & 'airline') & target (i.e. 'airline_sentiment') of dataframe.
twitter_df[['text', 'airline', 'airline_sentiment']].head(10)

,text,airline,airline_sentiment
8817,@JetBlue @ProfessorpaUL15 Always happy to help!,Delta,positive
14299,"As am I, @AmericanAir - but thankfully there w...",American,negative
8039,@JetBlue Now ur asking for the heavy guns! You...,Delta,neutral
2856,@united thanks for having ground crews that ar...,United,negative
821,"@united yes in Lusaka, Zambia. My guess is the...",United,negative
9582,@USAirways will all flights out of DFW be Canc...,US Airways,neutral
5150,@SouthwestAir baggage claim has already change...,Southwest,negative
7569,"@JetBlue after applying my new tag, she left m...",Delta,negative
14636,@AmericanAir leaving over 20 minutes Late Flig...,American,negative
3157,@united Surprised to go from 1K last year to n...,United,neutral


Here, in feature column '**text**', we have some non-ASCII characters (e.g. emoji) like ™, œ. So we clean-up our text data using following function: 

In [0]:
def clean_non_ascii(text):
    """
    Function to remove all non-ASCII characters (e.g. emoji) from given text.
    :param text: Text of review from user in data.
    :return: Text string with removed all non-ASCII characters from it.
    """
    return ''.join(i for i in text if ord(i) < 128)

In [0]:
# Let's apply this function to our 'text' feature column in dataframe.
twitter_df['text'] = twitter_df['text'].apply(func=clean_non_ascii)

We create four dataframes respectively for: 
**Train features**, **Train targets**, **Test features**, **Test targets**. <br />
That's because to separate training & testing sets.

In [0]:
train_features = pd.DataFrame()
train_targets = pd.DataFrame()
test_features = pd.DataFrame()
test_targets = pd.DataFrame()

# Take first 10000 records for training features & targets respectively.
train_features['text'] = twitter_df['text'].head(n=10000)
train_features['airline'] = twitter_df['airline'].head(n=10000)
train_targets['airline_sentiment'] = twitter_df['airline_sentiment'].head(n=10000)

# Similarly features & targets above 10000 records will be for test.
test_features['text'] = twitter_df['text'].iloc[len(train_features.index):]
test_features['airline'] = twitter_df['airline'].iloc[len(train_features.index):]
test_targets['airline_sentiment'] = twitter_df['airline_sentiment'].iloc[len(train_targets.index):]

Now inspect how many **airline** companies & unique classes **airline_sentiment** are present using `describe()` method:

In [0]:
print(" {*} Details of 'airline' feature column: \n", twitter_df['airline'].describe(), \
      "\n\n {*} Details of 'airline_sentiment' target column: \n", twitter_df['airline_sentiment'].describe())

 {*} Details of 'airline' feature column: 
 count      14640
unique         6
top       United
freq        3822
Name: airline, dtype: object 

 {*} Details of 'airline_sentiment' target column: 
 count        14640
unique           3
top       negative
freq          9178
Name: airline_sentiment, dtype: object


So here '**`unique`**' section of both results shows the actual unique records in those dataframe columns. It means **6** companies of '**airline**' (feature) & **3** classes of '**airline_sentiment**' (target).

We have '**airline**' (feature) & '**airline_sentiment**' (target) in text form. So let's use ***One-hot encoding*** to convert string format to processable numeric format. <br/> We first convert string to integer and then integer to one-hot encoding.

In [0]:
# First use 'LabelEncoder()' in Scikit-learn to convert from string to integer then use 'to_categorical()' of Keras to get respective one-hot encoding.
# One-hot encoding for training set 'airline' feature:
label_encoder_train_airline = LabelEncoder()
train_airline_feature_int = label_encoder_train_airline.fit_transform(y=train_features['airline'])
train_airline_feature_encoded = tf.keras.utils.to_categorical(y=train_airline_feature_int, 
                                                              num_classes=train_features['airline'].describe()['unique'])

# One-hot encoding for test set 'airline' feature:
label_encoder_test_airline = LabelEncoder()
test_airline_feature_int = label_encoder_test_airline.fit_transform(y=test_features['airline'])
test_airline_feature_encoded = tf.keras.utils.to_categorical(y=test_airline_feature_int, 
                                                             num_classes=test_features['airline'].describe()['unique'])

# Conversion for training targets 'airline_sentiment':
label_encoder_train_targets = LabelEncoder()
train_targets_int = label_encoder_train_targets.fit_transform(y=train_targets['airline_sentiment'])
train_targets_encoded = tf.keras.utils.to_categorical(y=train_targets_int, 
                                                      num_classes=train_targets['airline_sentiment'].describe()['unique'])

# Similarly conversion for test targets 'airline_sentiment':
label_encoder_test_targets = LabelEncoder()
test_targets_int = label_encoder_test_targets.fit_transform(y=test_targets['airline_sentiment'])
test_targets_encoded = tf.keras.utils.to_categorical(y=test_targets_int, 
                                                     num_classes=test_targets['airline_sentiment'].describe()['unique'])

# Let's see how encoding is done.
print(' {*} Oirginal train targets: \n', train_targets['airline_sentiment'].head(5), \
     '\n\n {*} One-hot encoded train targets: \n', [train_targets_encoded[i] for i in range(5)])

 {*} Oirginal train targets: 
 8817     positive
14299    negative
8039      neutral
2856     negative
821      negative
Name: airline_sentiment, dtype: object 

 {*} One-hot encoded train targets: 
 [array([0., 0., 1.], dtype=float32), array([1., 0., 0.], dtype=float32), array([0., 1., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32)]


The feature column '**text**' has textual setences so we use `Tokenizer` that vectorize collection of written text into sequence of integers. That means we use **word embedding**. <br />First we create an object of `Tokenizer` & define a function that fits tokenizer on given text and then return the sequence of integers for that text:

In [0]:
# Create Tokenizer object.
tokenizer = tf.keras.preprocessing.text.Tokenizer()

def get_tokenizer_encoding(text=""):
  """
  Function to get sequence of integers for given text.
  :param text: Text of review from user in data.
  :return: Sequence or vector of integers for given text.
  """
  tokenizer.fit_on_texts(texts=text)
  return tokenizer.texts_to_sequences(texts=text)

In [0]:
# Transform and get tokenize encoded text features.
train_text_feature_tokenize_encoded = get_tokenizer_encoding(text=train_features['text'])
test_text_feature_tokenize_encoded = get_tokenizer_encoding(text=test_features['text'])

# Let's check how the sequence is encoded for first sentence in text data.
print(' Original sentence: ', train_features['text'][0], '\n Encoded sequence: ', train_features_tokenize_encoded[0])

 Original sentence:  @VirginAmerica What @dhepburn said. 
 Encoded sequence:  [12, 71, 115, 3, 11524, 1501, 94, 82, 172, 2504]


We have to get maximum sequence length of encoded text between train & test data, to make all sequences of equal length.
<br/>Then use `pad_sequences()` of Keras in order to get every encoded text sequence having the same length. Since we can't use sequences of different lengths for processing.

In [0]:
# Initially get maximum length of encoded text between train & test encoded data.
max_length_text = len(max(train_features_tokenize_encoded, key=len)) \
    if len(max(train_features_tokenize_encoded, key=len)) > len(max(test_features_tokenize_encoded, key=len)) \
    else len(max(test_features_tokenize_encoded, key=len))
print(' Maximum length of encoded text sequence: ', max_length_text)

 Maximum length of encoded text sequence:  35


In [0]:
# Get equal sequence lengths using 'pad_sequences()' for train & test sets.
train_text_feature_padded = tf.keras.preprocessing.sequence.pad_sequences(sequences=train_text_feature_tokenize_encoded,
                                                                          truncating='pre', padding='post',
                                                                          maxlen=max_length_text)

test_text_feature_padded = tf.keras.preprocessing.sequence.pad_sequences(sequences=test_text_feature_tokenize_encoded,
                                                                         truncating='pre', padding='post',
                                                                         maxlen=max_length_text)
print(' {*} Padded sequences with equal shapes ', train_features_padded[0].shape, ' each (training features): \n')
print(train_features_padded)

 {*} Padded sequences with equal shapes  (35,)  each (training features): 

[[ 12  71 115 ...   0   0   0]
 [ 16 234   9 ...   0   0   0]
 [ 13  93  40 ...   0   0   0]
 ...
 [ 16 320 688 ...   0   0   0]
 [ 13  38  75 ...   0   0   0]
 [ 13  57 140 ...   0   0   0]]


Now we have to get total vocabulary size which is necessary for embedding (first) layer in the analysis model.

In [0]:
# Get total vocabulary size using 'word_index' property of tokenizer object which is required for embedding layer in our model.
vocabulary_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index.
print(' The vocabulary size for our data is: ', vocabulary_size)

 The vocabulary size for our data is:  15769


Next we'll define our model to train & test on data. <br />
Model architecture: <br />
**1. Embedding layer** -> This is the first layer in our model since we have text as word embeddings in feature. So it is important to learn those embeddings. <br />
**2. LSTM** -> Long Short-Term Memory are hidden layers that can maintain their state, which allows to handle sentences where next word depends on previous. Our model's feautre is textual sentences (with word embeddings) so we provided two LSTMs with 100 memory cells each since more memory cells & deeper network may get better results. <br />
**3. Dense** -> Dense is fully connected layer which connects 100 neurons of LSTM in hidden layer to interpret the features extracted from sequence. At the end, another Dense layer with 'softmax' activation is used with 3 neurons to get result among the possible three classes of target (i.e. negative, neutral or positive).

In [0]:
# Define model.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocabulary_size, input_length=max_length_text,
                                    output_dim=50))
model.add(tf.keras.layers.LSTM(units=100, return_sequences=True))
model.add(tf.keras.layers.LSTM(units=100))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

print(' {*} Model architecture: \n')
print(model.summary())

# Compile model.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

 {*} Model architecture: 

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 35, 50)            796700    
_________________________________________________________________
lstm_12 (LSTM)               (None, 35, 100)           60400     
_________________________________________________________________
lstm_13 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 303       
Total params: 947,903
Trainable params: 947,903
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print('\n\n Training model... \n')
model.fit(x=[train_text_feature_padded, train_airline_feature_encoded], y=train_targets_encoded,
          batch_size=64, epochs=10, steps_per_epoch=100)
print('\n\n Training finished... \n')



 Training model... 

Epoch 1/10
100/100 [==============================] - 1077s 11s/step - loss: 0.0476 - acc: 0.9832
Epoch 2/10
100/100 [==============================] - 1073s 11s/step - loss: 0.0447 - acc: 0.9880
Epoch 3/10
100/100 [==============================] - 1075s 11s/step - loss: 0.0161 - acc: 0.9962
Epoch 4/10
100/100 [==============================] - 1075s 11s/step - loss: 0.0123 - acc: 0.9966
Epoch 5/10
100/100 [==============================] - 1076s 11s/step - loss: 0.0110 - acc: 0.9965
Epoch 6/10
100/100 [==============================] - 1075s 11s/step - loss: 0.0097 - acc: 0.9966
Epoch 7/10
100/100 [==============================] - 1077s 11s/step - loss: 0.0798 - acc: 0.9736
Epoch 8/10
100/100 [==============================] - 1093s 11s/step - loss: 0.0664 - acc: 0.9787
Epoch 9/10
100/100 [==============================] - 1098s 11s/step - loss: 0.0218 - acc: 0.9925
Epoch 10/10
100/100 [==============================] - 1096s 11s/step - loss: 0.0153 - acc: 0.9

After training, we'll evaluate model on test data:

In [0]:
# Evaluate model on test data.
loss_acc = model.evaluate(x=[test_text_feature_padded, test_airline_feature_encoded], y=test_targets_encoded, verbose=1)
print(' Test data:= Loss: %0.6f, Accuracy: %0.2f%%' % (loss_acc[0], loss_acc[1] * 100))

4640/4640 [==============================] - 3s 702us/sample - loss: 1.0975 - acc: 0.6265
 Test data:= Loss: 1.097461, Accuracy: 62.65%


Our model can correctly classify  more than half of given data (62.65%) on training with just 10000 records. <br />
This can be improved significantly by providing more training data to the model, which will increase its vocabulary size also. Another way may be using set of pre-built embeddings such as **GloVe ("global vectors for word representation")**, which is constructed using the text of Wikipedia. <br />
So here, by creating embeddings on the fly our model performed good on test data with limited set of training.